Let us first do some dimension reduction of the word vector.

In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed')
answer = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv",sep='\t')

这里是我的错,当时没考虑到这里没有标签不方便

In [3]:
gap_test["A-coref"] = answer["A-coref"]
gap_test["B-coref"] = answer["B-coref"]

In [4]:
gap_test.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL,vector,...,B_pos,pron_pos,A_idx,B_idx,pron_idx,A_vector,B_vector,pron_vector,A-coref,B-coref
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...,"[[-0.30144718, -0.08635384, -0.20015159, -0.25...",...,0.608696,0.826087,"[54, 55, 56, 57, 58, 59]","[61, 62, 63]",[81],"[[0.14136104, -0.38602263, -0.19260919, 0.4973...","[[0.6761073, -0.45650303, -0.27585956, -0.3139...","[[-0.1408648, -0.009012785, -0.58574224, -0.96...",True,False
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie,"[[0.34895068, 0.6811254, 0.92398053, -0.417637...",...,0.333333,0.380952,"[63, 64, 65]","[68, 69, 70, 71, 72]",[78],"[[-0.75297713, -0.91880596, -0.31332776, 0.235...","[[-0.014572581, 0.3896196, -0.4235178, -0.2201...","[[-0.5127397, 0.38019675, -0.19245711, -0.6960...",True,False
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...,"[[-0.3016174, 0.06391026, -0.15569207, -0.1670...",...,0.505155,0.556701,"[43, 44]","[59, 60, 61, 62]",[65],"[[-0.44509542, -0.38031942, 0.01635306, 0.1235...","[[0.06467356, 0.14759275, 0.38993636, -0.09622...","[[-0.40798748, 0.8667205, -0.14823496, 0.31704...",False,True
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band),"[[0.31311065, -0.4944703, 0.44825977, -0.12736...",...,0.657143,0.619048,[40],"[75, 76, 77, 78, 79, 80]",[71],"[[0.043780833, 0.44859782, 0.566811, -0.007530...","[[-0.40079117, 0.6081313, 0.008237168, -0.2687...","[[-0.23520903, -0.029176882, -0.047637727, 0.0...",False,True
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera,"[[0.1210996, 0.19799706, 0.4914001, -0.4640768...",...,0.619565,0.923913,"[59, 60, 61, 62, 63]","[78, 79]",[116],"[[0.31352907, 0.8938127, -0.27805957, 0.610802...","[[0.60271496, 0.43095493, -0.86851233, 0.58121...","[[-0.7814366, -0.24526855, 0.24610633, -0.3730...",False,True


In [5]:
train_vector_ = np.concatenate(gap_train.vector,axis = 0)

In [6]:
mean_vector = train_vector_.mean(axis = 0)
train_vector = train_vector_ - mean_vector

In [7]:
train_vector.shape

(223174, 1024)

In [8]:
import torch
import torch.nn as nn


dim = 1024
batch_size = 256
learning_rate = 1e-3
steps = 5000

In [9]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Linear(dim, 128,bias = False)
        self.decoder = nn.Linear(128, dim,bias = False)

    def forward(self, x):
        h = self.encoder(x)
        x_ = self.decoder(h)
        return x_,h,x-x_

In [10]:
PPA = autoencoder()
PPA.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PPA.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,train_vector.shape[0] - batch_size+1)
    data_train = torch.Tensor(train_vector[pos:pos+batch_size,:]).cuda()
    # ===================forward=====================
    data_reconstruction, _,_ = PPA(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA.state_dict(), './temp_result/PPA.pth')

steps [1/5000], loss:0.3948
steps [2/5000], loss:0.3439
steps [3/5000], loss:0.3521
steps [4/5000], loss:0.3698
steps [5/5000], loss:0.3549
steps [6/5000], loss:0.3455
steps [7/5000], loss:0.3429
steps [8/5000], loss:0.3194
steps [9/5000], loss:0.3220
steps [10/5000], loss:0.3847
steps [11/5000], loss:0.3524
steps [12/5000], loss:0.3212
steps [13/5000], loss:0.3294
steps [14/5000], loss:0.3153
steps [15/5000], loss:0.3732
steps [16/5000], loss:0.3087
steps [17/5000], loss:0.2915
steps [18/5000], loss:0.3046
steps [19/5000], loss:0.3427
steps [20/5000], loss:0.3459
steps [21/5000], loss:0.3268
steps [22/5000], loss:0.3029
steps [23/5000], loss:0.2993
steps [24/5000], loss:0.3176
steps [25/5000], loss:0.3150
steps [26/5000], loss:0.3166
steps [27/5000], loss:0.3005
steps [28/5000], loss:0.2724
steps [29/5000], loss:0.2827
steps [30/5000], loss:0.2973
steps [31/5000], loss:0.3043
steps [32/5000], loss:0.2791
steps [33/5000], loss:0.2662
steps [34/5000], loss:0.2557
steps [35/5000], loss:0

steps [378/5000], loss:0.1604
steps [379/5000], loss:0.1394
steps [380/5000], loss:0.1759
steps [381/5000], loss:0.1591
steps [382/5000], loss:0.1375
steps [383/5000], loss:0.1519
steps [384/5000], loss:0.1642
steps [385/5000], loss:0.1442
steps [386/5000], loss:0.1598
steps [387/5000], loss:0.1619
steps [388/5000], loss:0.1540
steps [389/5000], loss:0.1630
steps [390/5000], loss:0.1558
steps [391/5000], loss:0.1505
steps [392/5000], loss:0.1565
steps [393/5000], loss:0.1627
steps [394/5000], loss:0.1511
steps [395/5000], loss:0.1745
steps [396/5000], loss:0.1485
steps [397/5000], loss:0.1642
steps [398/5000], loss:0.1725
steps [399/5000], loss:0.1529
steps [400/5000], loss:0.1363
steps [401/5000], loss:0.1405
steps [402/5000], loss:0.1708
steps [403/5000], loss:0.1574
steps [404/5000], loss:0.1451
steps [405/5000], loss:0.1505
steps [406/5000], loss:0.1459
steps [407/5000], loss:0.1458
steps [408/5000], loss:0.1494
steps [409/5000], loss:0.1585
steps [410/5000], loss:0.1427
steps [411

steps [759/5000], loss:0.1522
steps [760/5000], loss:0.1594
steps [761/5000], loss:0.1601
steps [762/5000], loss:0.1843
steps [763/5000], loss:0.1493
steps [764/5000], loss:0.1675
steps [765/5000], loss:0.1527
steps [766/5000], loss:0.1413
steps [767/5000], loss:0.1704
steps [768/5000], loss:0.1360
steps [769/5000], loss:0.1504
steps [770/5000], loss:0.1555
steps [771/5000], loss:0.1449
steps [772/5000], loss:0.1728
steps [773/5000], loss:0.1566
steps [774/5000], loss:0.1499
steps [775/5000], loss:0.1396
steps [776/5000], loss:0.1482
steps [777/5000], loss:0.1462
steps [778/5000], loss:0.1399
steps [779/5000], loss:0.1371
steps [780/5000], loss:0.1662
steps [781/5000], loss:0.1471
steps [782/5000], loss:0.1584
steps [783/5000], loss:0.1671
steps [784/5000], loss:0.1594
steps [785/5000], loss:0.1423
steps [786/5000], loss:0.1455
steps [787/5000], loss:0.1993
steps [788/5000], loss:0.1463
steps [789/5000], loss:0.1444
steps [790/5000], loss:0.1378
steps [791/5000], loss:0.1611
steps [792

steps [1144/5000], loss:0.1456
steps [1145/5000], loss:0.1593
steps [1146/5000], loss:0.1396
steps [1147/5000], loss:0.1588
steps [1148/5000], loss:0.1450
steps [1149/5000], loss:0.1588
steps [1150/5000], loss:0.1524
steps [1151/5000], loss:0.1641
steps [1152/5000], loss:0.1455
steps [1153/5000], loss:0.1463
steps [1154/5000], loss:0.1356
steps [1155/5000], loss:0.1571
steps [1156/5000], loss:0.1548
steps [1157/5000], loss:0.1559
steps [1158/5000], loss:0.1579
steps [1159/5000], loss:0.1531
steps [1160/5000], loss:0.1490
steps [1161/5000], loss:0.1393
steps [1162/5000], loss:0.1355
steps [1163/5000], loss:0.1264
steps [1164/5000], loss:0.1419
steps [1165/5000], loss:0.1464
steps [1166/5000], loss:0.1434
steps [1167/5000], loss:0.1435
steps [1168/5000], loss:0.1619
steps [1169/5000], loss:0.1498
steps [1170/5000], loss:0.1627
steps [1171/5000], loss:0.1451
steps [1172/5000], loss:0.1572
steps [1173/5000], loss:0.1478
steps [1174/5000], loss:0.1613
steps [1175/5000], loss:0.1462
steps [1

steps [1530/5000], loss:0.1566
steps [1531/5000], loss:0.1267
steps [1532/5000], loss:0.1350
steps [1533/5000], loss:0.1363
steps [1534/5000], loss:0.1324
steps [1535/5000], loss:0.1310
steps [1536/5000], loss:0.1846
steps [1537/5000], loss:0.1612
steps [1538/5000], loss:0.1405
steps [1539/5000], loss:0.1530
steps [1540/5000], loss:0.1460
steps [1541/5000], loss:0.1535
steps [1542/5000], loss:0.1425
steps [1543/5000], loss:0.1470
steps [1544/5000], loss:0.1545
steps [1545/5000], loss:0.1400
steps [1546/5000], loss:0.1315
steps [1547/5000], loss:0.1384
steps [1548/5000], loss:0.1487
steps [1549/5000], loss:0.1632
steps [1550/5000], loss:0.1546
steps [1551/5000], loss:0.1455
steps [1552/5000], loss:0.1448
steps [1553/5000], loss:0.1415
steps [1554/5000], loss:0.1707
steps [1555/5000], loss:0.1537
steps [1556/5000], loss:0.1549
steps [1557/5000], loss:0.1749
steps [1558/5000], loss:0.1517
steps [1559/5000], loss:0.1384
steps [1560/5000], loss:0.1370
steps [1561/5000], loss:0.1283
steps [1

...

steps [3102/5000], loss:0.1477
steps [3103/5000], loss:0.1359
steps [3104/5000], loss:0.1495
steps [3105/5000], loss:0.1395
steps [3106/5000], loss:0.1459
steps [3107/5000], loss:0.1352
steps [3108/5000], loss:0.1415
steps [3109/5000], loss:0.1409
steps [3110/5000], loss:0.1344
steps [3111/5000], loss:0.1462
steps [3112/5000], loss:0.1555
steps [3113/5000], loss:0.1502
steps [3114/5000], loss:0.1617
steps [3115/5000], loss:0.1482
steps [3116/5000], loss:0.1458
steps [3117/5000], loss:0.1423
steps [3118/5000], loss:0.1555
steps [3119/5000], loss:0.1484
steps [3120/5000], loss:0.1436
steps [3121/5000], loss:0.1453
steps [3122/5000], loss:0.1548
steps [3123/5000], loss:0.1279
steps [3124/5000], loss:0.1553
steps [3125/5000], loss:0.1328
steps [3126/5000], loss:0.1517
steps [3127/5000], loss:0.1431
steps [3128/5000], loss:0.1338
steps [3129/5000], loss:0.1759
steps [3130/5000], loss:0.1580
steps [3131/5000], loss:0.1637
steps [3132/5000], loss:0.1385
steps [3133/5000], loss:0.1529
steps [3

steps [3509/5000], loss:0.1248
steps [3510/5000], loss:0.1407
steps [3511/5000], loss:0.1576
steps [3512/5000], loss:0.1533
steps [3513/5000], loss:0.1430
steps [3514/5000], loss:0.1494
steps [3515/5000], loss:0.1451
steps [3516/5000], loss:0.1589
steps [3517/5000], loss:0.1397
steps [3518/5000], loss:0.1320
steps [3519/5000], loss:0.1663
steps [3520/5000], loss:0.1457
steps [3521/5000], loss:0.1499
steps [3522/5000], loss:0.1341
steps [3523/5000], loss:0.1395
steps [3524/5000], loss:0.1484
steps [3525/5000], loss:0.1456
steps [3526/5000], loss:0.1569
steps [3527/5000], loss:0.1423
steps [3528/5000], loss:0.1571
steps [3529/5000], loss:0.1414
steps [3530/5000], loss:0.1395
steps [3531/5000], loss:0.1456
steps [3532/5000], loss:0.1403
steps [3533/5000], loss:0.1291
steps [3534/5000], loss:0.1340
steps [3535/5000], loss:0.1513
steps [3536/5000], loss:0.1599
steps [3537/5000], loss:0.1492
steps [3538/5000], loss:0.1476
steps [3539/5000], loss:0.1458
steps [3540/5000], loss:0.1450
steps [3

steps [3888/5000], loss:0.1385
steps [3889/5000], loss:0.1401
steps [3890/5000], loss:0.1592
steps [3891/5000], loss:0.1469
steps [3892/5000], loss:0.1364
steps [3893/5000], loss:0.1285
steps [3894/5000], loss:0.1405
steps [3895/5000], loss:0.1365
steps [3896/5000], loss:0.1546
steps [3897/5000], loss:0.1580
steps [3898/5000], loss:0.1673
steps [3899/5000], loss:0.1539
steps [3900/5000], loss:0.1419
steps [3901/5000], loss:0.1396
steps [3902/5000], loss:0.1416
steps [3903/5000], loss:0.1568
steps [3904/5000], loss:0.1383
steps [3905/5000], loss:0.1433
steps [3906/5000], loss:0.1500
steps [3907/5000], loss:0.1599
steps [3908/5000], loss:0.1411
steps [3909/5000], loss:0.1461
steps [3910/5000], loss:0.1584
steps [3911/5000], loss:0.1586
steps [3912/5000], loss:0.1418
steps [3913/5000], loss:0.1531
steps [3914/5000], loss:0.1774
steps [3915/5000], loss:0.1619
steps [3916/5000], loss:0.1228
steps [3917/5000], loss:0.1391
steps [3918/5000], loss:0.1325
steps [3919/5000], loss:0.1445
steps [3

steps [4275/5000], loss:0.1554
steps [4276/5000], loss:0.1429
steps [4277/5000], loss:0.1440
steps [4278/5000], loss:0.1328
steps [4279/5000], loss:0.1555
steps [4280/5000], loss:0.1514
steps [4281/5000], loss:0.1346
steps [4282/5000], loss:0.1506
steps [4283/5000], loss:0.1839
steps [4284/5000], loss:0.1542
steps [4285/5000], loss:0.1485
steps [4286/5000], loss:0.1356
steps [4287/5000], loss:0.1361
steps [4288/5000], loss:0.1537
steps [4289/5000], loss:0.1591
steps [4290/5000], loss:0.1379
steps [4291/5000], loss:0.1424
steps [4292/5000], loss:0.1465
steps [4293/5000], loss:0.1391
steps [4294/5000], loss:0.1797
steps [4295/5000], loss:0.1720
steps [4296/5000], loss:0.1577
steps [4297/5000], loss:0.1493
steps [4298/5000], loss:0.1503
steps [4299/5000], loss:0.1625
steps [4300/5000], loss:0.1580
steps [4301/5000], loss:0.1403
steps [4302/5000], loss:0.1659
steps [4303/5000], loss:0.1556
steps [4304/5000], loss:0.1450
steps [4305/5000], loss:0.1504
steps [4306/5000], loss:0.1488
steps [4

steps [4669/5000], loss:0.1535
steps [4670/5000], loss:0.1478
steps [4671/5000], loss:0.1344
steps [4672/5000], loss:0.1305
steps [4673/5000], loss:0.1428
steps [4674/5000], loss:0.1466
steps [4675/5000], loss:0.1608
steps [4676/5000], loss:0.1335
steps [4677/5000], loss:0.1435
steps [4678/5000], loss:0.1381
steps [4679/5000], loss:0.1474
steps [4680/5000], loss:0.1660
steps [4681/5000], loss:0.1365
steps [4682/5000], loss:0.1420
steps [4683/5000], loss:0.1375
steps [4684/5000], loss:0.1446
steps [4685/5000], loss:0.1434
steps [4686/5000], loss:0.1327
steps [4687/5000], loss:0.1307
steps [4688/5000], loss:0.1416
steps [4689/5000], loss:0.1558
steps [4690/5000], loss:0.1580
steps [4691/5000], loss:0.1521
steps [4692/5000], loss:0.1340
steps [4693/5000], loss:0.1439
steps [4694/5000], loss:0.1248
steps [4695/5000], loss:0.1601
steps [4696/5000], loss:0.1441
steps [4697/5000], loss:0.1274
steps [4698/5000], loss:0.1521
steps [4699/5000], loss:0.1657
steps [4700/5000], loss:0.1456
steps [4

In [12]:
for df in [gap_train,gap_valid,gap_test]:
    for i in range(df.shape[0]):
        _,_,first_ppa_vector = PPA(torch.Tensor(df.vector[i]-mean_vector).cuda())
        first_ppa_vector = first_ppa_vector.cpu().data.numpy()
        df.loc[i,"vector"]  = [first_ppa_vector]
        df.loc[i,"A_vector"] = [first_ppa_vector[df.A_idx[i],:]]
        df.loc[i,"B_vector"] = [first_ppa_vector[df.B_idx[i],:]]
        df.loc[i,"pron_vector"] = [first_ppa_vector[df.pron_idx[i],:]]

In [13]:
gap_train.to_pickle('./temp_result/train_kaggle_processed_PPA')
gap_test.to_pickle('./temp_result/test_kaggle_processed_PPA')
gap_valid.to_pickle('./temp_result/valid_kaggle_processed_PPA')